In [1]:
import cv2
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [4]:
def extract_keypoints(frame, proto_file, weights_file, threshold, model_name, BODY_PARTS):
    net = cv2.dnn.readNetFromCaffe(proto_file, weights_file)

    image_height, image_width = frame.shape[:2]

    input_blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (image_width, image_height), (0, 0, 0), swapRB=False, crop=False)
    net.setInput(input_blob)

    out = net.forward()
    out_height = out.shape[2]
    out_width = out.shape[3]

    frame_height, frame_width = frame.shape[:2]

    points = []

    for i in range(len(BODY_PARTS)):
        prob_map = out[0, i, :, :]
        min_val, prob, min_loc, point = cv2.minMaxLoc(prob_map)

        x = (frame_width * point[0]) / out_width
        x = int(x)
        y = (frame_height * point[1]) / out_height
        y = int(y)

        points.append((x, y))

    return points

In [2]:
protoFile_mpi = "./openpose-master/models/pose/mpi/pose_deploy_linevec.prototxt"
weightsFile_mpi = "./openpose-master/models/pose/mpi/pose_iter_160000.caffemodel"

BODY_PARTS_MPI = {
    0: "Head",
    1: "Neck",
    2: "RShoulder",
    3: "RElbow",
    4: "RWrist",
    5: "LShoulder",
    6: "LElbow",
    7: "LWrist",
    8: "RHip",
    9: "RKnee",
    10: "RAnkle",
    11: "LHip",
    12: "LKnee",
    13: "LAnkle",
    14: "Chest",
    15: "Background"
}

In [16]:
threshold = 0.2

scaler = StandardScaler()

model = LogisticRegression()

X_train = pd.DataFrame()

scaler_fitted = False

cap = cv2.VideoCapture(0)  
while True:
    ret, frame = cap.read()

    keypoints = extract_keypoints(frame, protoFile_mpi, weightsFile_mpi, threshold, "MPI", BODY_PARTS_MPI)

    if keypoints:
        data = pd.DataFrame(columns=["BodyPart", "X", "Y"])

        for part_index, (x, y) in enumerate(keypoints):
            body_part = BODY_PARTS_MPI.get(part_index, "Unknown")
            data = data.append({"BodyPart": body_part, "X": x, "Y": y}, ignore_index=True)

        data_pivoted = data.pivot(index="BodyPart", columns="X", values="Y")

        if "RShoulder" in data_pivoted.columns and "LShoulder" in data_pivoted.columns:
            data_pivoted["Shoulder_Asymmetry"] = abs(data_pivoted["RShoulder"] - data_pivoted["LShoulder"])
        else:
            data_pivoted["Shoulder_Asymmetry"] = np.nan

        selected_columns = ["Head_X", "Head_Y", "Neck_X", "Neck_Y", "RShoulder_X", "RShoulder_Y", "LShoulder_X", "LShoulder_Y"]

        data_selected = pd.DataFrame(columns=selected_columns)

        for part in selected_columns:
            x_col = part + "_X"
            y_col = part + "_Y"
    
            if x_col in data.columns and y_col in data.columns:
                data_selected[part] = data[x_col]
                data_selected[part + "_Y"] = data[y_col]

        data_selected["Shoulder_Asymmetry"] = abs(data_selected["RShoulder_X"] - data_selected["LShoulder_X"])

        X_train = pd.concat([X_train, data_selected])

        print("X_train shape:", X_train.shape)

        if not scaler_fitted and X_train.shape[0] >= 10:  
            scaler.fit(X_train)
            print("StandardScaler fitted!")
            scaler_fitted = True  

        X = data_selected

        if scaler_fitted:  
            X_scaled = scaler.transform(X)
            prediction = model.predict(X_scaled)

    cv2.imshow("Shoulder Asymmetry Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\rmady\AppData\Local\Temp\ipykernel_16980\1155713350.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"BodyPart": body_part, "X": x, "Y": y}, ignore_index=True)
C:\Users\rmady\AppData\Local\Temp\ipykernel_16980\1155713350.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"BodyPart": body_part, "X": x, "Y": y}, ignore_index=True)
C:\Users\rmady\AppData\Local\Temp\ipykernel_16980\1155713350.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"BodyPart": body_part, "X": x, "Y": y}, ignore_index=True)
C:\Users\rmady\AppData\Local\Temp\ipykernel_16980\1155713350.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas i

X_train shape: (0, 9)


C:\Users\rmady\AppData\Local\Temp\ipykernel_16980\1155713350.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"BodyPart": body_part, "X": x, "Y": y}, ignore_index=True)
C:\Users\rmady\AppData\Local\Temp\ipykernel_16980\1155713350.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"BodyPart": body_part, "X": x, "Y": y}, ignore_index=True)
C:\Users\rmady\AppData\Local\Temp\ipykernel_16980\1155713350.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"BodyPart": body_part, "X": x, "Y": y}, ignore_index=True)
C:\Users\rmady\AppData\Local\Temp\ipykernel_16980\1155713350.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas i

X_train shape: (0, 9)


C:\Users\rmady\AppData\Local\Temp\ipykernel_16980\1155713350.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"BodyPart": body_part, "X": x, "Y": y}, ignore_index=True)
C:\Users\rmady\AppData\Local\Temp\ipykernel_16980\1155713350.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"BodyPart": body_part, "X": x, "Y": y}, ignore_index=True)
C:\Users\rmady\AppData\Local\Temp\ipykernel_16980\1155713350.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"BodyPart": body_part, "X": x, "Y": y}, ignore_index=True)
C:\Users\rmady\AppData\Local\Temp\ipykernel_16980\1155713350.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas i

X_train shape: (0, 9)


C:\Users\rmady\AppData\Local\Temp\ipykernel_16980\1155713350.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"BodyPart": body_part, "X": x, "Y": y}, ignore_index=True)
C:\Users\rmady\AppData\Local\Temp\ipykernel_16980\1155713350.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"BodyPart": body_part, "X": x, "Y": y}, ignore_index=True)
C:\Users\rmady\AppData\Local\Temp\ipykernel_16980\1155713350.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"BodyPart": body_part, "X": x, "Y": y}, ignore_index=True)
C:\Users\rmady\AppData\Local\Temp\ipykernel_16980\1155713350.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas i

X_train shape: (0, 9)
